# Mini Dollar Strategy - Backtesting and Optimization

This notebook demonstrates:
1. Strategy backtesting
2. Performance analysis
3. Parameter optimization
4. Results visualization

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from itertools import product

from src.agents.coordinator import StrategyCoordinator
from src.reporting.performance import PerformanceAnalyzer

# Configure plotting
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 6]
sns.set_palette('Set2')

## 1. Initial Strategy Test

In [ ]:
# Initialize components
coordinator = StrategyCoordinator(
    initial_balance=100000,
    max_position=1,
    stop_loss=100,
    take_profit=200
)

analyzer = PerformanceAnalyzer()

# Run backtest
end_date = datetime.now()
start_date = end_date - timedelta(days=180)

results = coordinator.backtest(
    start_date=start_date.strftime('%Y-%m-%d'),
    end_date=end_date.strftime('%Y-%m-%d'),
    timeframe='5T'
)

# Get performance metrics
metrics = analyzer.calculate_metrics(results)
print("\nPerformance Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.2f}")

## 2. Detailed Performance Analysis

In [ ]:
# Generate detailed performance report
report = analyzer.generate_report(results)
print(report)

# Plot equity curve and drawdowns
analyzer.plot_equity_curve(results)

def plot_trade_distribution(results):
    """Plot distribution of trade profits/losses."""
    trades = results[results['trade_executed']]['profit']
    
    plt.figure(figsize=(15, 5))
    sns.histplot(trades, bins=50)
    plt.title('Trade Profit/Loss Distribution')
    plt.xlabel('Points')
    plt.grid(True)
    plt.show()
    
    print("\nTrade Statistics:")
    print(f"Average Trade: {trades.mean():.2f}")
    print(f"Std Dev: {trades.std():.2f}")
    print(f"Best Trade: {trades.max():.2f}")
    print(f"Worst Trade: {trades.min():.2f}")

plot_trade_distribution(results)

## 3. Parameter Optimization

In [ ]:
def optimize_parameters(start_date, end_date):
    """Grid search for optimal parameters."""
    # Parameter ranges
    rsi_periods = [9, 14, 21]
    ma_fast_periods = [5, 9, 13]
    ma_slow_periods = [21, 34, 55]
    stop_losses = [50, 100, 150]
    take_profits = [100, 200, 300]
    
    results = []
    
    # Grid search
    for params in product(rsi_periods, ma_fast_periods, ma_slow_periods, stop_losses, take_profits):
        rsi, ma_fast, ma_slow, sl, tp = params
        
        # Initialize strategy with parameters
        coord = StrategyCoordinator(
            initial_balance=100000,
            max_position=1,
            stop_loss=sl,
            take_profit=tp
        )
        
        # Run backtest
        backtest_results = coord.backtest(
            start_date=start_date,
            end_date=end_date,
            timeframe='5T'
        )
        
        # Calculate metrics
        metrics = analyzer.calculate_metrics(backtest_results)
        
        results.append({
            'rsi_period': rsi,
            'ma_fast': ma_fast,
            'ma_slow': ma_slow,
            'stop_loss': sl,
            'take_profit': tp,
            'total_profit': metrics['total_profit'],
            'win_rate': metrics['win_rate'],
            'sharpe_ratio': metrics['sharpe_ratio'],
            'max_drawdown': metrics['max_drawdown']
        })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Run optimization
end_date = datetime.now()
start_date = end_date - timedelta(days=90)  # 3 months for optimization
    
optimization_results = optimize_parameters(
    start_date=start_date.strftime('%Y-%m-%d'),
    end_date=end_date.strftime('%Y-%m-%d')
)
    
# Show top 10 parameter combinations
print("Top 10 Parameter Combinations by Total Profit:")
print(optimization_results.sort_values('total_profit', ascending=False).head(10))

## 4. Analysis of Optimal Parameters

In [ ]:
def analyze_parameter_impact(results_df):
    """Analyze the impact of different parameters on performance."""
    # RSI Period Analysis
    plt.figure(figsize=(15, 5))
    sns.boxplot(x='rsi_period', y='total_profit', data=results_df)
    plt.title('RSI Period vs Total Profit')
    plt.show()
    
    # Moving Averages Analysis
    plt.figure(figsize=(15, 5))
    sns.scatterplot(x='ma_fast', y='ma_slow', size='total_profit', 
                    hue='win_rate', data=results_df)
    plt.title('Moving Averages Periods vs Performance')
    plt.show()
    
    # Stop Loss and Take Profit Analysis
    plt.figure(figsize=(15, 5))
    sns.scatterplot(x='stop_loss', y='take_profit', size='total_profit',
                    hue='sharpe_ratio', data=results_df)
    plt.title('Stop Loss vs Take Profit')
    plt.show()

# Analyze parameter impact
analyze_parameter_impact(optimization_results)

# Get best parameters
best_params = optimization_results.loc[optimization_results['total_profit'].idxmax()]
print("\nBest Parameters:")
print(f"RSI Period: {best_params['rsi_period']}")
print(f"Fast MA: {best_params['ma_fast']}")
print(f"Slow MA: {best_params['ma_slow']}")
print(f"Stop Loss: {best_params['stop_loss']}")
print(f"Take Profit: {best_params['take_profit']}")

## 5. Out-of-Sample Testing

In [ ]:
# Test optimized strategy on recent data
coordinator_optimized = StrategyCoordinator(
    initial_balance=100000,
    max_position=1,
    stop_loss=best_params['stop_loss'],
    take_profit=best_params['take_profit']
)

# Run backtest on recent data
end_date = datetime.now()
start_date = end_date - timedelta(days=30)  # Last month

recent_results = coordinator_optimized.backtest(
    start_date=start_date.strftime('%Y-%m-%d'),
    end_date=end_date.strftime('%Y-%m-%d'),
    timeframe='5T'
)

# Calculate and show performance
recent_metrics = analyzer.calculate_metrics(recent_results)
print("\nOut-of-Sample Performance:")
for metric, value in recent_metrics.items():
    print(f"{metric}: {value:.2f}")

# Plot equity curve
analyzer.plot_equity_curve(recent_results)